In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import import_ipynb
import Captcha_seg
import time

importing Jupyter notebook from Captcha_seg.ipynb


In [2]:
CATEGORIES =[]
for i in os.listdir("captcha_segmented/"):
    CATEGORIES.append(i)
print(CATEGORIES)

['6', 'F', 'J', 'T', 'E', '3', 'U', 'K', 'M', '4', 'X', 'W', 'D', 'Z', 'V', 'H', '8', 'C', 'L', 'S', 'P', 'A', 'R', '1', '5', 'Q', 'N', '7', 'G', '2', 'Y', 'B', '9']


In [3]:
SEGMENTS_DIR = 'Segmented'
CAPTCHA_DIR = 'test/'
predictions = []
b_captcha = []

DATADIR = "./captcha_segmented/"
#CATEGORIES= ["2","3","4","5","6","7","8","b","c","d","e","f","g","m","n","p","w","x","y"]
for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        #plt.imshow(img_array, cmap="gray")
        break
    break

In [4]:
print(img_array.shape)

(45, 19)


In [5]:
IMG_SIZE = 50

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
#plt.imshow(new_array, cmap= 'gray')
#plt.show()

In [6]:
train_data = []

def create_train_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category) 
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                train_data.append([new_array,class_num])
            except Exception as e:
                pass
            
create_train_data()

In [7]:
print(len(train_data))

2352


In [8]:
import random

random.shuffle(train_data)

In [9]:
for sample in train_data[:10]:
    print(sample[1])

16
26
14
11
0
30
17
6
26
21


In [10]:
X = []
y = []

In [11]:
for features, label in train_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [12]:
import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [13]:
pickle_in= open("X.pickle", "rb")
X = pickle.load(pickle_in)
X.shape[1:]

(50, 50, 1)

In [15]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten, Conv2D, MaxPooling2D

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X = X/255.0

model = Sequential()

model.add(Conv2D(64,(3,3),input_shape= X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))


model.add(Dense(33))
model.add(Activation("softmax"))

from keras.utils import to_categorical
y_binary = to_categorical(y)

model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model.fit(X,y_binary,batch_size=128, epochs=20, validation_split = 0.2)


Train on 1881 samples, validate on 471 samples
Epoch 1/20
1881/1881 [==============================] - 17s 9ms/step - loss: 3.4952 - acc: 0.0335 - val_loss: 3.5072 - val_acc: 0.0212
Epoch 2/20
1881/1881 [==============================] - 13s 7ms/step - loss: 3.4883 - acc: 0.0372 - val_loss: 3.4981 - val_acc: 0.0510
Epoch 3/20
1881/1881 [==============================] - 13s 7ms/step - loss: 3.4747 - acc: 0.0553 - val_loss: 3.4961 - val_acc: 0.0531
Epoch 4/20
1881/1881 [==============================] - 13s 7ms/step - loss: 3.4259 - acc: 0.0670 - val_loss: 3.4155 - val_acc: 0.0786
Epoch 5/20
1881/1881 [==============================] - 13s 7ms/step - loss: 3.2753 - acc: 0.1106 - val_loss: 3.2847 - val_acc: 0.1189
Epoch 6/20
1881/1881 [==============================] - 13s 7ms/step - loss: 3.0296 - acc: 0.1691 - val_loss: 3.0682 - val_acc: 0.1656
Epoch 7/20
1881/1881 [==============================] - 13s 7ms/step - loss: 2.7313 - acc: 0.2605 - val_loss: 2.9054 - val_acc: 0.2293
Epoch 8/

In [16]:
loss_value, metrics_value = model.evaluate(X,y_binary,batch_size=10)

2352/2352 [==============================] - 5s 2ms/step


In [17]:
print(loss_value, metrics_value)

0.9222098876323018 0.7559523796345912


In [18]:
def similarity(a,b):
    i =0
    for x,y in zip(a,b):
        if x==y:
            i+= 1
    percentage = (i/5) * 100
    return percentage

def prepare(filepath):
    IMG_SIZE = 50
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
    #plt.imshow(new_array,'gray')
    return new_array.reshape(-1, IMG_SIZE,IMG_SIZE,1)

total_time = 0
score = 0
count = 0
for captchas in os.listdir(CAPTCHA_DIR):
    start_time = time.time()
    predict_list= []
    captcha_name = captchas.split('.')
    print('\n')
    print('Actual: ',captcha_name[0])
    
    captcha_path =os.path.join(CAPTCHA_DIR,captchas)
    Captcha_seg.segment(captcha_path)
    
    files = []
    for segments in os.listdir(SEGMENTS_DIR):
        files.append(segments)
        files.sort()
    
    for segments in files:
        filepath = os.path.join(SEGMENTS_DIR,segments)
        prediction = model.predict([prepare(filepath)])
        predict_list.append(CATEGORIES[np.argmax(prediction[0])])
    
    elapsed_time = time.time()-start_time
    total_time += elapsed_time
    final_prediction = ''.join(predict_list)
    if captcha_name[0]== final_prediction:
        score +=1
    
    print('Predicted:',final_prediction,'\n')
    print('Percentage Similarity= ',similarity(captcha_name[0],final_prediction))
    print('Elapsed time= ',elapsed_time,'s')
    print('<------------------------------->')    
    predictions.append(similarity(captcha_name[0],final_prediction))
    count += 1


print('Total number of CAPTCHAS broken: ',score)    
print('Total number of CAPTCHAS attempted: ',count)
avg_prediction = score/count
print('Average Characters Predicted= ',avg_prediction,'%')
print('Total time taken= ',total_time,'s')
#model = tf.keras.models.load_model('captcha_alpha.model')
#prediction = model.predict([prepare('3.png')])
#print(len(prediction[0]))
#print(prediction[0])
#print(CATEGORIES[np.argmax(prediction[0])])



Actual:  BPU1Q
Predicted: UHUVL 

Percentage Similarity=  20.0
Elapsed time=  4.237852334976196 s
<------------------------------->


Actual:  NNTP3
Predicted: JJJHL 

Percentage Similarity=  0.0
Elapsed time=  0.020899534225463867 s
<------------------------------->


Actual:  VBJP
Predicted: VGJQL 

Percentage Similarity=  40.0
Elapsed time=  0.016901254653930664 s
<------------------------------->


Actual:  8JRJV
Predicted: QJXJL 

Percentage Similarity=  40.0
Elapsed time=  0.024805545806884766 s
<------------------------------->


Actual:  34ESN
Predicted: SVUQL 

Percentage Similarity=  0.0
Elapsed time=  0.01449894905090332 s
<------------------------------->


Actual:  B1Q96
Predicted: UUQGL 

Percentage Similarity=  20.0
Elapsed time=  0.014552116394042969 s
<------------------------------->


Actual:  7ATR6
Predicted: 7QJ1L 

Percentage Similarity=  20.0
Elapsed time=  0.023355484008789062 s
<------------------------------->


Actual:  QTKD9
Predicted: CJJJL 

Percentage S

Predicted: H2HJL 

Percentage Similarity=  40.0
Elapsed time=  0.03394746780395508 s
<------------------------------->


Actual:  UXN5V
Predicted: JXJQL 

Percentage Similarity=  20.0
Elapsed time=  0.02908611297607422 s
<------------------------------->


Actual:  N1R8
Predicted: JJQLL 

Percentage Similarity=  0.0
Elapsed time=  0.023485183715820312 s
<------------------------------->


Actual:  1ZEJ1
Predicted: J7LJL 

Percentage Similarity=  20.0
Elapsed time=  0.04012656211853027 s
<------------------------------->


Actual:  CKN36
Predicted: ULLQL 

Percentage Similarity=  0.0
Elapsed time=  0.0200502872467041 s
<------------------------------->


Actual:  H3DF
Predicted: UFLHL 

Percentage Similarity=  0.0
Elapsed time=  0.038059234619140625 s
<------------------------------->


Actual:  TWDW
Predicted: JWDVL 

Percentage Similarity=  40.0
Elapsed time=  0.02481698989868164 s
<------------------------------->


Actual:  C7HS
Predicted: Q7LQL 

Percentage Similarity=  20.0
Elapse

Predicted: 7XWGL 

Percentage Similarity=  40.0
Elapsed time=  0.015468597412109375 s
<------------------------------->


Actual:  Q4RW
Predicted: QJJVL 

Percentage Similarity=  20.0
Elapsed time=  0.016669750213623047 s
<------------------------------->


Actual:  UKWD
Predicted: UKVQL 

Percentage Similarity=  40.0
Elapsed time=  0.016731977462768555 s
<------------------------------->


Actual:  CNR7
Predicted: QVHHL 

Percentage Similarity=  0.0
Elapsed time=  0.015169620513916016 s
<------------------------------->


Actual:  86SK
Predicted: QQGKL 

Percentage Similarity=  20.0
Elapsed time=  0.014537811279296875 s
<------------------------------->


Actual:  F8DY
Predicted: HQDYL 

Percentage Similarity=  40.0
Elapsed time=  0.024346351623535156 s
<------------------------------->


Actual:  VJE9
Predicted: VJJSL 

Percentage Similarity=  40.0
Elapsed time=  0.014560461044311523 s
<------------------------------->


Actual:  LYJP
Predicted: L1JCL 

Percentage Similarity=  40.0
E

Predicted: JPHXL 

Percentage Similarity=  40.0
Elapsed time=  0.016266822814941406 s
<------------------------------->


Actual:  RYN3
Predicted: JYVHL 

Percentage Similarity=  20.0
Elapsed time=  0.01865077018737793 s
<------------------------------->


Actual:  V5D4
Predicted: VUUJL 

Percentage Similarity=  20.0
Elapsed time=  0.014688491821289062 s
<------------------------------->


Actual:  ZZUY
Predicted: 7HJYL 

Percentage Similarity=  20.0
Elapsed time=  0.014228343963623047 s
<------------------------------->


Actual:  MGVH
Predicted: JGVHL 

Percentage Similarity=  60.0
Elapsed time=  0.014609098434448242 s
<------------------------------->


Actual:  8NJYZ
Predicted: QQJVL 

Percentage Similarity=  20.0
Elapsed time=  0.014264583587646484 s
<------------------------------->


Actual:  RSE93
Predicted: HQGSL 

Percentage Similarity=  0.0
Elapsed time=  0.014503240585327148 s
<------------------------------->


Actual:  XG2PM
Predicted: XQXHL 

Percentage Similarity=  20.0

Predicted: JU1VL 

Percentage Similarity=  0.0
Elapsed time=  0.01678013801574707 s
<------------------------------->


Actual:  JZ8J7
Predicted: J7JJL 

Percentage Similarity=  40.0
Elapsed time=  0.01779651641845703 s
<------------------------------->


Actual:  NHU9
Predicted: JHJLL 

Percentage Similarity=  20.0
Elapsed time=  0.014491796493530273 s
<------------------------------->


Actual:  H2886
Predicted: LGVQL 

Percentage Similarity=  0.0
Elapsed time=  0.014295578002929688 s
<------------------------------->


Actual:  23BJ
Predicted: GQD1L 

Percentage Similarity=  0.0
Elapsed time=  0.01490640640258789 s
<------------------------------->


Actual:  F8EV
Predicted: SQGVL 

Percentage Similarity=  20.0
Elapsed time=  0.014662027359008789 s
<------------------------------->


Actual:  UBYHR
Predicted: JDVHL 

Percentage Similarity=  20.0
Elapsed time=  0.01510000228881836 s
<------------------------------->


Actual:  A6ZD
Predicted: 4G7JL 

Percentage Similarity=  0.0
Elaps

Predicted: WG77L 

Percentage Similarity=  20.0
Elapsed time=  0.01652216911315918 s
<------------------------------->


Actual:  WKURC
Predicted: VK11L 

Percentage Similarity=  20.0
Elapsed time=  0.025496959686279297 s
<------------------------------->


Actual:  G2RC
Predicted: CYRJL 

Percentage Similarity=  20.0
Elapsed time=  0.015304327011108398 s
<------------------------------->


Actual:  728XN
Predicted: 7QUXL 

Percentage Similarity=  40.0
Elapsed time=  0.015210151672363281 s
<------------------------------->


Actual:  45UU
Predicted: 45LVL 

Percentage Similarity=  40.0
Elapsed time=  0.015381097793579102 s
<------------------------------->


Actual:  2R45
Predicted: 7Q45L 

Percentage Similarity=  40.0
Elapsed time=  0.014456033706665039 s
<------------------------------->


Actual:  K4F1
Predicted: NJH1L 

Percentage Similarity=  20.0
Elapsed time=  0.014827728271484375 s
<------------------------------->


Actual:  3LYL
Predicted: GJYLL 

Percentage Similarity=  40.0

Predicted: VQFSL 

Percentage Similarity=  0.0
Elapsed time=  0.015929460525512695 s
<------------------------------->


Actual:  27EMS
Predicted: 27EVL 

Percentage Similarity=  60.0
Elapsed time=  0.01790308952331543 s
<------------------------------->


Actual:  XTZ66
Predicted: XJ7QL 

Percentage Similarity=  20.0
Elapsed time=  0.017615795135498047 s
<------------------------------->


Actual:  YF44N
Predicted: VK4UL 

Percentage Similarity=  20.0
Elapsed time=  0.015402078628540039 s
<------------------------------->


Actual:  4SEV
Predicted: 4GUVL 

Percentage Similarity=  40.0
Elapsed time=  0.015182733535766602 s
<------------------------------->


Actual:  ALVME
Predicted: ALVNL 

Percentage Similarity=  60.0
Elapsed time=  0.01555943489074707 s
<------------------------------->


Actual:  GZ2ZZ
Predicted: QCQ2L 

Percentage Similarity=  0.0
Elapsed time=  0.01570725440979004 s
<------------------------------->


Actual:  HM4U
Predicted: JJJUL 

Percentage Similarity=  20.0


Predicted: XVUGL 

Percentage Similarity=  20.0
Elapsed time=  0.041345834732055664 s
<------------------------------->


Actual:  UY23
Predicted: UV4FL 

Percentage Similarity=  20.0
Elapsed time=  0.023908376693725586 s
<------------------------------->


Actual:  V1BG
Predicted: VJRJL 

Percentage Similarity=  20.0
Elapsed time=  0.09337139129638672 s
<------------------------------->


Actual:  6BWB
Predicted: 6QVLL 

Percentage Similarity=  20.0
Elapsed time=  0.04137229919433594 s
<------------------------------->


Actual:  CSUP
Predicted: QCULL 

Percentage Similarity=  20.0
Elapsed time=  0.023854494094848633 s
<------------------------------->


Actual:  AZ5L
Predicted: A7SLL 

Percentage Similarity=  40.0
Elapsed time=  0.01520228385925293 s
<------------------------------->


Actual:  EU5YD
Predicted: HLDWL 

Percentage Similarity=  0.0
Elapsed time=  0.01845836639404297 s
<------------------------------->


Actual:  E73T3
Predicted: J7GJL 

Percentage Similarity=  20.0
Ela

Predicted: KVS7L 

Percentage Similarity=  20.0
Elapsed time=  0.015798330307006836 s
<------------------------------->


Actual:  UM5E
Predicted: UV11L 

Percentage Similarity=  20.0
Elapsed time=  0.018828868865966797 s
<------------------------------->


Actual:  JL7TB
Predicted: JLJQL 

Percentage Similarity=  40.0
Elapsed time=  0.014485359191894531 s
<------------------------------->


Actual:  SM8Q
Predicted: JJAJL 

Percentage Similarity=  0.0
Elapsed time=  0.014698266983032227 s
<------------------------------->


Actual:  7JNW
Predicted: 7JNWL 

Percentage Similarity=  80.0
Elapsed time=  0.014624357223510742 s
<------------------------------->


Actual:  THHDV
Predicted: JJJ1L 

Percentage Similarity=  0.0
Elapsed time=  0.014580726623535156 s
<------------------------------->


Actual:  BB29A
Predicted: GG2QL 

Percentage Similarity=  20.0
Elapsed time=  0.015145301818847656 s
<------------------------------->


Actual:  2KV6
Predicted: 2NVGL 

Percentage Similarity=  40.0

Predicted: UQJJL 

Percentage Similarity=  0.0
Elapsed time=  0.01884937286376953 s
<------------------------------->


Actual:  BH26K
Predicted: FUGQL 

Percentage Similarity=  0.0
Elapsed time=  0.019034147262573242 s
<------------------------------->
Total number of CAPTCHAS broken:  0
Total number of CAPTCHAS attempted:  588
Average Characters Predicted=  0.0 %
Total time taken=  15.390063047409058 s
